In [1]:
# data science
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import datetime


# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, text, inspect, func

In [2]:
# Create engine using the `spacex.sqlite` database file
engine = create_engine("sqlite:///tornadoes_clean.sqlite")

In [3]:
# INSPECT to confirm existence

# Create the inspector and connect it to the engine
inspector_gadget = inspect(engine)

# Collect the names of tables within the database
tables = inspector_gadget.get_table_names()

# print metadata for each table
for table in tables:
    print(table)
    print("-----------")
    
    # get columns
    columns = inspector_gadget.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])

    print()

tornadoes
-----------
id BIGINT
tornado_id BIGINT
yr BIGINT
month BIGINT
date TEXT
state TEXT
category FLOAT
injuries BIGINT
fatalities BIGINT
loss FLOAT
start_lat FLOAT
start_longitude FLOAT
end_latitude FLOAT
end_longitude FLOAT
distance_traveled FLOAT
width BIGINT
seasons TEXT



In [9]:
# Bar Query

# User inputs
user_seasons = "All"

if user_seasons != 'All':
    where_clause = "seasons = :user_seasons"
else:
    where_clause = "1 = 1"  # This will always be true and effectively ignore the WHERE clause

# Query
query = f"""
    SELECT
        yr,
        category,
        injuries,
        fatalities
    FROM
        tornadoes
    WHERE
        {where_clause}
    ORDER BY
        category ASC;
"""

# execute query
with engine.connect() as connection:
    df_bar = pd.read_sql(text(query), con=engine.connect())

df_bar.head()

,yr,category,injuries,fatalities
0,2016,0.0,0,0
1,2016,0.0,0,0
2,2016,0.0,0,0
3,2016,0.0,0,0
4,2016,0.0,0,0


In [8]:
# Pie Query

# User Input
if user_seasons != 'All':
    where_clause = f"seasons LIKE '{user_seasons}'"
else:
    where_clause = "1 = 1"

# Query
query = f"""
    SELECT
        loss,
        category
    FROM
        tornadoes
    WHERE
        {where_clause};
"""

# Convert data into dictionary
df_pie = pd.read_sql(text(query), con=engine.connect())
df_pie.head()

,loss,category
0,0.0,0.0
1,0.0,0.0
2,1000000.0,2.0
3,0.0,0.0
4,0.0,0.0


In [10]:
# Table Query

# User Input
if user_seasons != 'All':
    where_clause = f"seasons LIKE '{user_seasons}'"
else:
    where_clause = "1 = 1"

# Query
query = f"""
    SELECT
        yr,
        seasons,
        state,
        category,
        injuries,
        fatalities,
        loss
    FROM
        tornadoes
    WHERE
        {where_clause}
    ORDER BY
        category DESC;
"""

# Convert data into dictionary
df_table = pd.read_sql(text(query), con=engine.connect())
df_table.head()

,yr,seasons,state,category,injuries,fatalities,loss
0,2016,spring,KS,4.0,8,0,0.0
1,2016,spring,OK,4.0,0,1,1000000.0
2,2017,winter,MO,4.0,12,1,14800000.0
3,2017,spring,TX,4.0,25,2,1200000.0
4,2019,spring,AL,4.0,90,23,1750000.0


In [7]:
# Map Query

# User Inputs
user_year = "All"
where_clause = f"yr = {user_year}"

if user_year != 'All':
    where_clause = f"yr = {user_year}"
else:
    where_clause = f"yr > 2000"

# Query
query = f"""
    SELECT
        yr,
        state,
        category,
        loss,
        start_lat,
        start_longitude,
        end_latitude,
        end_longitude,
        distance_traveled,
        width
    FROM
        tornadoes
    WHERE
        {where_clause}
"""

# execute query
df_map = pd.read_sql(text(query), con=engine.connect())
df_map.head()

,yr,state,category,loss,start_lat,start_longitude,end_latitude,end_longitude,distance_traveled,width
0,2016,IN,0.0,0.0,41.6421,-86.3046,41.6557,-86.2886,1.25,30
1,2016,IN,0.0,0.0,41.6685,-86.2307,41.6689,-86.2300,0.05,40
2,2016,OR,2.0,1000000.0,45.7200,-123.9400,45.7200,-123.9400,0.70,225
3,2016,OR,0.0,0.0,45.4700,-123.9700,45.4700,-123.9700,0.01,10
4,2016,KS,0.0,0.0,39.2482,-97.4066,39.2482,-97.4066,0.10,40
